<a href="https://colab.research.google.com/github/MirianGreiner/Previsao_motivo_viagem/blob/main/Previs%C3%A3o_do_motivo_de_viagem_do_passageiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>